In [60]:
import numpy as np
import pandas as pd


In [61]:
import gzip

with gzip.open("goodreads_books.json.gz") as f:
    line = f.readline()

In [62]:
line

b'{"isbn": "0312853122", "text_reviews_count": "1", "series": [], "country_code": "US", "language_code": "", "popular_shelves": [{"count": "3", "name": "to-read"}, {"count": "1", "name": "p"}, {"count": "1", "name": "collection"}, {"count": "1", "name": "w-c-fields"}, {"count": "1", "name": "biography"}], "asin": "", "is_ebook": "false", "average_rating": "4.00", "kindle_asin": "", "similar_books": [], "description": "", "format": "Paperback", "link": "https://www.goodreads.com/book/show/5333265-w-c-fields", "authors": [{"author_id": "604031", "role": ""}], "publisher": "St. Martin\'s Press", "num_pages": "256", "publication_day": "1", "isbn13": "9780312853129", "publication_month": "9", "edition_information": "", "publication_year": "1984", "url": "https://www.goodreads.com/book/show/5333265-w-c-fields", "image_url": "https://images.gr-assets.com/books/1310220028m/5333265.jpg", "book_id": "5333265", "ratings_count": "3", "work_id": "5400751", "title": "W.C. Fields: A Life on Film", "t

In [63]:
import json 

dt = json.loads(line)

In [64]:
dt

{'isbn': '0312853122',
 'text_reviews_count': '1',
 'series': [],
 'country_code': 'US',
 'language_code': '',
 'popular_shelves': [{'count': '3', 'name': 'to-read'},
  {'count': '1', 'name': 'p'},
  {'count': '1', 'name': 'collection'},
  {'count': '1', 'name': 'w-c-fields'},
  {'count': '1', 'name': 'biography'}],
 'asin': '',
 'is_ebook': 'false',
 'average_rating': '4.00',
 'kindle_asin': '',
 'similar_books': [],
 'description': '',
 'format': 'Paperback',
 'link': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'authors': [{'author_id': '604031', 'role': ''}],
 'publisher': "St. Martin's Press",
 'num_pages': '256',
 'publication_day': '1',
 'isbn13': '9780312853129',
 'publication_month': '9',
 'edition_information': '',
 'publication_year': '1984',
 'url': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'image_url': 'https://images.gr-assets.com/books/1310220028m/5333265.jpg',
 'book_id': '5333265',
 'ratings_count': '3',
 'work_id': '5400751',
 'title': '

In [6]:
#'book_id':
#'title_without_series': 'W.C. Fields: A Life on Film'}
#'url':
#'ratings_count':
#'image_url':


In [65]:
def parse_fields(line):
    data = json.loads(line)
    return {
        "book_id": data["book_id"], 
        "title": data["title_without_series"], 
        "rating": data["ratings_count"], 
        "url": data["url"], 
        "cover_image": data["image_url"]
    }

In [66]:
parse_fields(line)

{'book_id': '5333265',
 'title': 'W.C. Fields: A Life on Film',
 'rating': '3',
 'url': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'cover_image': 'https://images.gr-assets.com/books/1310220028m/5333265.jpg'}

In [67]:
books_titles = []
with gzip.open("goodreads_books.json.gz") as f:
    while True:
        line = f.readline()
        if not line:
            break
        fields = parse_fields(line)
        
        try:
            ratings = int(fields["rating"])
        except ValueError:
            continue
            
        if ratings > 5:
            books_titles.append(fields)

In [68]:
import pandas as pd

titles = pd.DataFrame.from_dict(books_titles)

In [69]:
titles["rating"] = pd.to_numeric(titles["rating"])
titles["mod_title"] = titles["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True)
titles["mod_title"] = titles["mod_title"].str.lower()
titles["mod_title"] = titles["mod_title"].str.replace("\s+", " ", regex=True)
titles = titles[titles["mod_title"].str.len() > 0]
titles.to_json("books_titles.json")

In [12]:
titles.values[10]

array(['89371', 'The Te Of Piglet', 11,
       'https://www.goodreads.com/book/show/89371.The_Te_Of_Piglet',
       'https://s.gr-assets.com/assets/nophoto/book/111x148-bcc042a9c91a29c1d680899eff700a03.png',
       'the te of piglet'], dtype=object)

In [81]:
liked_books = ["4408", "31147619", "29983711"]

In [86]:
book_lists = pd.DataFrame(books_titles)

In [87]:
b_list = book_lists.to_dict()

In [94]:
book_lists['title'].values


array(['Good Harbor',
       'The Unschooled Wizard (Sun Wolf and Starhawk, #1-2)',
       'Best Friends Forever', ...,
       "The Children's Classic Poetry Collection",
       '101 Nights: Volume One (101 Nights, #1-3)',
       "The Spanish Duke's Virgin Bride (Innocent Mistress, Virgin Bride, #1) (Harlequin Presents, #2679)"],
      dtype=object)

In [14]:
liked_books

['4408', '31147619', '29983711']

In [15]:
book_mapping = {}

with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        book_mapping[csv_id] = book_id

In [18]:

overlap_users = set()

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            continue

        try:
            rating = int(rating)  
        except ValueError:
            continue
        
        book_id = book_mapping[csv_id]
        
        if book_id in liked_books and rating >= 4:
                overlap_users.add(user_id) 

ValueError: not enough values to unpack (expected 5, got 1)

In [20]:
rec_lines = []

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            book_id = book_mapping[csv_id]
            rec_lines.append([user_id, book_id, rating])

ValueError: not enough values to unpack (expected 5, got 1)

In [21]:
import pandas as pd

recs = pd.DataFrame(rec_lines, columns=["user_id", "book_id", "rating"])
recs["book_id"] = recs["book_id"].astype(str)

In [22]:
recs

,user_id,book_id,rating
0,284,977284,3
1,284,890054,4
2,284,837153,3
3,284,1586480,4
4,284,41814,5
...,...,...,...
345641,130537,28503788,0
345642,130537,1774836,0
345643,130537,29983711,5
345644,130537,29864343,0


In [23]:
top_recs = recs["book_id"].value_counts().head(10)

In [25]:
top_recs = top_recs.index.values

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [26]:

# books_titles = pd.read_json("books_titles.json")
# books_titles["book_id"] = books_titles["book_id"].astype(str)

In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [ ]:
books_titles[books_titles["book_id"].isin(top_recs)]

In [ ]:
all_recs = recs["book_id"].value_counts()

In [ ]:
all_recs = all_recs.to_frame().reset_index()
all_recs.columns = ["book_id", "book_count"]

In [ ]:
all_recs.head(5)

In [ ]:
all_recs = all_recs.merge(books_titles, how="inner", on="book_id")

In [ ]:
all_recs["score"] = all_recs["book_count"] * (all_recs["book_count"] / all_recs["rating_x"])


In [ ]:
all_recs.sort_values("score", ascending=False).head(10)

In [ ]:
all_recs[all_recs["book_count"] > 50].sort_values("score", ascending=False).head(10)


In [ ]:
popular_recs = all_recs[all_recs["book_count"] > 50].sort_values("score", ascending=False).head(10)

In [ ]:
populer = pd.DataFrame(popular_recs, columns=["title","url","cover_image"])

In [ ]:
def make_clickable(val):    
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)


populer[~popular_recs["book_id"].isin(liked_books)].head(10).style.format({'url': make_clickable, 'cover_image': show_image})
 